# AI Fairness 360 on COMPAS

This notebook shows use of the [AI Fairness 360 Toolkit](https://aif360.mybluemix.net/) on the COMPAS dataset.

This notebook is based on examples in [AI Fairness 360 examples](https://github.com/IBM/AIF360/blob/master/examples).

For ML fairness background on COMPAS see:
- https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing
- https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
- http://www.crj.org/assets/2017/07/9_Machine_bias_rejoinder.pdf

The dataset is from the [COMPAS kaggle page](https://www.kaggle.com/danofer/compass).

In this notebook, we check for bias in the initial training data, mitigate the bias, and recheck.

We use the COMPAS dataset, splitting it into a training and test dataset.  Then we look for bias in the creation of a machine learning model to predict if a person belongs in the "Low" risk class according to COMPAS (negative class), or the "Medium" or "High" risk class (positive class).  The protected attribute is "Race", with "1" (for Caucasian) and "0"(for Not Caucasian) being the values for the privileged and unprivileged groups, respectively.

Note: The label value 0 in this case is considered favorable (no recidivism).

Here are the steps involved:
#### Step 1: Write import statements
#### Step 2: Set bias detection options, load dataset, and split between train and test
#### Step 3: Compute fairness metric on original training dataset
#### Step 4: Mitigate bias by transforming the original dataset
#### Step 5: Compute fairness metric on transformed training dataset


### Step 1: Import Statements
As with any python program, the first step will be to import the necessary packages.  Below we import several components from the `aif360` package.  We import the CompasDataset, metrics to check for bias, and classes related to the algorithm we will use to mitigate bias.

In [1]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

### Step 2: Load dataset, specifying protected attribute, and split dataset into train and test
In Step 2 we load the initial dataset, setting the protected attribute to be race.  We then split the original dataset into training and testing datasets.  Although we will use only the training dataset in this tutorial, a normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model.  Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the race attribute.  These are key inputs for detecting and mitigating bias, which will be Step 3 and Step 4.  

In [2]:
dataset_orig = CompasDataset(protected_attribute_names=['race'], # this dataset also contains protected
                                                                 # attribute for "sex" ('Female', 'Male')
                                                                 # which we do not consider in this evaluation
                             privileged_classes=[['Caucasian']], # race == 'Caucasian' is considered privileged
                             features_to_drop=['sex'],           # ignore notably sex-related attributes
                             metadata={'label_maps': [{0.0: 'No recid.', 1.0: 'Did recid.'}],
                                       'protected_attribute_maps': [{0.0: 'Not Caucasian', 1.0: 'Caucasian'}]})

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

### Step 3: Compute fairness metric on original training dataset
Now that we've identified the protected attribute 'race' and defined privileged and unprivileged values, we can use aif360 to detect bias in the dataset.  One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter.   A negative value indicates less favorable outcomes for the unprivileged groups.  This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class.  The code below performs this check and displays the output, showing that the difference is -0.086690.

In [3]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.086690


### Step 4: Mitigate bias by transforming the original dataset
The previous step showed that the privileged group was getting 8% more favorable outcomes in the training dataset.   Since this is not desirable, we are going to try to mitigate this bias in the training dataset.  As stated above, this is called _pre-processing_ mitigation because it happens before the creation of the model.  

AI Fairness 360 implements several pre-processing mitigation algorithms.  We will choose the Reweighing algorithm [1], which is implemented in the `Reweighing` class in the `aif360.algorithms.preprocessing` package.  This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

`[1] F. Kamiran and T. Calders,  "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.`

In [4]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

### Step 5: Compute fairness metric on transformed dataset
Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3.  Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class.   We see the mitigation step was very effective, the difference in mean outcomes is now 0.0.  So we went from a 8% advantage for the privileged group to equality in terms of mean outcome.

In [5]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.000000
